In [0]:
# Testing for GPU availability
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
!pip install -q keras

### **To avoid tedious debates with the TA. Dina, we have to print all outputs of "each" step regardless of the importance of the step.** @Zyad


---











In [0]:
# Installing database file/s
!wget http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
!bzip2 -d 2016.04C.multisnr.pkl.bz2
!ls

In [0]:
# The classes Sequential and Dense are used to specify the nodes, connections, specifications of the neural network.
import keras
from keras.models import Sequential
from keras.layers import Dense

In [0]:
import pickle
import gzip
import numpy

with open('2016.04C.multisnr.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()
    print(p["QPSK"])

In [0]:
sum=0
for x in p.values():
  sum+=x.shape[0]
print(sum)
# for key,value in p.items():
#   if key[0]=='PAM4':
#     sum+=value.shape[0]
  
# print(sum)

In [0]:
print(type(p))
data = {}
allm = []
for k in p.keys():
  data[k[0]] = {}
  allm.append(k[0])
mod = sorted(set(allm))
print(mod)
print(data)
print(len(allm))


In [0]:
import numpy as np
from numpy import zeros, newaxis

for m in mod:
  for k in p.keys():
    if k[0] == m :
      for sig in range(len(p[k])):
        a = np.array(p[k][sig][0])[:, newaxis]
        b = np.array(p[k][sig][1])[:, newaxis]
        if k[1] not in data[k[0]]:
          data[k[0]][k[1]] = []
        data[k[0]][k[1]].append([a,b])

2 dimentions vectors because complex values a vector for real value and imaginary value

In [0]:

dat = []
#Training data and labels 
X = []
Y = []
#dictionary contating samples for each snr, key is snr and value is list of samples having this snr
samples_per_snr = {}
#dictonary accessed by snr and gives labels of mod types ,snr is index
labels_ofSamples_per_snr = {}
#dictionary saving all one-hot encodes of different modulation
mval = {}
count = 0

#changed variables z one_hot_label, x -> samples_per_snr , y->labels_ofSamples_per_snr  both used for testing

#loop all modulation techniques ex QPSK,BSK...etc
for m in mod:
  one_hot_label = np.zeros((len(mod),))
  one_hot_label[count] = 1     
  mval[m] = one_hot_label
  #repeat for each snr for modulation technique m
  for snr in data[m]:
    #print(m,snr)
    dat = data[m][snr]
    #train with 2/3 of data from start
    #1/3 for each channel ??
    for d in dat[:int(len(dat)//1.5)]:
      X.append(d)
      Y.append(one_hot_label)
      #loop the remaining 1/3 of data to test with them
      for d in dat[int(len(dat)//1.5):]:
        #only performed once to set the 20 snrs as keys in dictionary
        #each value for key is list of samples having this snrs
        if not snr in samples_per_snr:
          samples_per_snr[snr] = []
          labels_ofSamples_per_snr[snr] = []
        #the real loop body :
        samples_per_snr[snr].append(d)
        labels_ofSamples_per_snr[snr].append(one_hot_label)
  #increment loop counter
  count += 1 


In [0]:
print(len(X))
print(len(Y))

In [0]:
# Making the Confusion Matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(..., ...) 

In [0]:
'''
Sample Test
'''

input_dim_ = 30 #@param {type:"integer"}

# Initialising the ANN
classifier = Sequential()

# Initialising input layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = input_dim_))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])